In [1]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
from torch import nn
from torch.utils import data
from torchvision import transforms

import training_utils as utils
import distillation_methods_module

import numpy as np

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

cuda:0


In [2]:
# Download the training and test sets
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.FashionMNIST(
    root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.FashionMNIST(
    root="../data", train=False, transform=trans, download=True)

train_iter = data.DataLoader(mnist_train, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
test_iter = data.DataLoader(mnist_test, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

In [3]:
class Teacher_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Linear(784, 1200)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

        self.linear_2 = nn.Linear(1200, 1200)

        self.linear_3 = nn.Linear(1200, 1200)

        self.linear_4 = nn.Linear(1200, 10)
    
    def forward(self, input):
        input = torch.flatten(input, start_dim=1)
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_4(out)

        return out

In [13]:
class Student_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Linear(784, 600)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

        self.linear_2 = nn.Linear(600, 1200)

        self.linear_3 = nn.Linear(1200, 1200)

        self.linear_4 = nn.Linear(1200, 10)
    
    def forward(self, input):
        input = torch.flatten(input, start_dim=1)
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_4(out)

        return out

In [5]:
teacher = Teacher_Net().to(device)
num_epochs = 10
loss_fn = nn.CrossEntropyLoss(reduction='none').to(device)
optimizer = torch.optim.Adam(teacher.parameters(), lr=0.001)
history = utils.train(teacher, utils.train_epoch, train_iter, test_iter, loss_fn, num_epochs, optimizer)

Epoch:		  0
train_metrics:	  (0.6198176907221477, 0.7714666666666666)
test_accuracy:	  0.8236
Epoch:		  1
train_metrics:	  (0.44270379422505696, 0.8398333333333333)
test_accuracy:	  0.854
Epoch:		  2
train_metrics:	  (0.40867470410664875, 0.85095)
test_accuracy:	  0.8562
Epoch:		  3
train_metrics:	  (0.3908185957590739, 0.8564333333333334)
test_accuracy:	  0.8641
Epoch:		  4
train_metrics:	  (0.3861622578620911, 0.8586333333333334)
test_accuracy:	  0.8673
Epoch:		  5
train_metrics:	  (0.37064978942871096, 0.8637666666666667)
test_accuracy:	  0.8739
Epoch:		  6
train_metrics:	  (0.3611437030792236, 0.8684166666666666)
test_accuracy:	  0.8724
Epoch:		  7
train_metrics:	  (0.35373455613454186, 0.8693)
test_accuracy:	  0.879
Epoch:		  8
train_metrics:	  (0.34425810718536376, 0.8747666666666667)
test_accuracy:	  0.8749
Epoch:		  9
train_metrics:	  (0.3427658058802287, 0.8749833333333333)
test_accuracy:	  0.8731


In [6]:
student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=5e-3)

d_logits = distillation_methods_module.Logits_Distiller(5, teacher=teacher, student=student, optimizer=optimizer)
d_logits.train(train_iter, test_iter, num_epochs)

Epoch:		  0
train_metrics:	  (1.896868804275679e-05, 0.7019666666666666)
test_accuracy:	  0.801
Epoch:		  1
train_metrics:	  (8.331771542240555e-06, 0.7899333333333334)
test_accuracy:	  0.8199
Epoch:		  2
train_metrics:	  (7.88387215191809e-06, 0.7968166666666666)
test_accuracy:	  0.8185
Epoch:		  3
train_metrics:	  (8.198371655695762e-06, 0.7983666666666667)
test_accuracy:	  0.8249
Epoch:		  4
train_metrics:	  (6.990285400145997e-06, 0.8082166666666667)
test_accuracy:	  0.8288
Epoch:		  5
train_metrics:	  (7.5786689568000535e-06, 0.8020666666666667)
test_accuracy:	  0.824
Epoch:		  6
train_metrics:	  (7.848203416991357e-06, 0.7991166666666667)
test_accuracy:	  0.8279
Epoch:		  7
train_metrics:	  (7.536703136671955e-06, 0.8046333333333333)
test_accuracy:	  0.8321
Epoch:		  8
train_metrics:	  (8.323327700297038e-06, 0.7936333333333333)
test_accuracy:	  0.8292
Epoch:		  9
train_metrics:	  (8.293020620476455e-06, 0.7955666666666666)
test_accuracy:	  0.8258


([0.7019666666666666,
  0.7899333333333334,
  0.7968166666666666,
  0.7983666666666667,
  0.8082166666666667,
  0.8020666666666667,
  0.7991166666666667,
  0.8046333333333333,
  0.7936333333333333,
  0.7955666666666666],
 [1.896868804275679e-05,
  8.331771542240555e-06,
  7.88387215191809e-06,
  8.198371655695762e-06,
  6.990285400145997e-06,
  7.5786689568000535e-06,
  7.848203416991357e-06,
  7.536703136671955e-06,
  8.323327700297038e-06,
  8.293020620476455e-06],
 [0.801,
  0.8199,
  0.8185,
  0.8249,
  0.8288,
  0.824,
  0.8279,
  0.8321,
  0.8292,
  0.8258])

In [14]:
student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)

d_features = distillation_methods_module.Features_Distiller(hint_layer=teacher.linear_2, guided_layer=student.linear_2, teacher=teacher, student=student, optimizer=optimizer)
d_features.train(train_iter, test_iter, num_epochs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (7168x28 and 784x600)

In [ ]:
student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)

hint_layers = (teacher.linear_2, teacher.linear_3)
guided_layers = (student.linear_2, student.linear_3)

d_relation = distillation_methods_module.Relations_Distiller(hint_layers=hint_layers, guided_layers=guided_layers, teacher=teacher, student=student, optimizer=optimizer)
d_relation.train(train_iter, test_iter, num_epochs)

Epoch:		  0
train_metrics:	  (218.26622141927083, 0.10048333333333333)
test_accuracy:	  0.0989
Epoch:		  1
train_metrics:	  (84.02734996744792, 0.10025)
test_accuracy:	  0.1
Epoch:		  2
train_metrics:	  (66.88911232096355, 0.10098333333333333)
test_accuracy:	  0.1
Epoch:		  3
train_metrics:	  (59.55752234700521, 0.10118333333333333)
test_accuracy:	  0.1
Epoch:		  4
train_metrics:	  (54.03042255859375, 0.10051666666666667)
test_accuracy:	  0.1
Epoch:		  5
train_metrics:	  (60.741406640625, 0.09818333333333333)
test_accuracy:	  0.1
Epoch:		  6
train_metrics:	  (50.79052355957031, 0.10016666666666667)
test_accuracy:	  0.1
Epoch:		  7
train_metrics:	  (47.62262318522136, 0.10048333333333333)
test_accuracy:	  0.1
Epoch:		  8
train_metrics:	  (46.51358276367188, 0.10021666666666666)
test_accuracy:	  0.1
Epoch:		  9
train_metrics:	  (46.01889170735677, 0.09996666666666666)
test_accuracy:	  0.1
Epoch:		  0
train_metrics:	  (1.44498934173584, 0.5841833333333334)
test_accuracy:	  0.7934
Epoch:		

([0.5841833333333334,
  0.7289166666666667,
  0.7474166666666666,
  0.7585166666666666,
  0.7671666666666667,
  0.7769666666666667,
  0.7821666666666667,
  0.7873666666666667,
  0.7937,
  0.7957166666666666],
 [1.44498934173584,
  0.9893265190124512,
  0.8613761878967285,
  0.7948878176371257,
  0.7473975632985433,
  0.712172764968872,
  0.6828827878316244,
  0.6606242468516031,
  0.6374113406499227,
  0.6198916083017985],
 [0.7934,
  0.8137,
  0.8229,
  0.8242,
  0.8248,
  0.8323,
  0.8347,
  0.8311,
  0.8348,
  0.8381])